#  Exercise 01
The goal is to find the best set of hyper-parameters which maximize the
performance on a training set.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from scipy.stats import expon, uniform
from scipy.stats import randint

df = pd.read_csv("https://www.openml.org/data/get_csv/1595261/adult-census.csv")
# Or use the local copy:
# df = pd.read_csv('../datasets/adult-census.csv')

target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=target_name)

df_train, df_test, target_train, target_test = train_test_split(
    data, target, random_state=42
)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


You should:
- create a preprocessor using an `OrdinalEncoder`
- use a `HistGradientBoostingClassifier` to make predictions
- use a `RandomizedSearchCV` to find the best set of hyper-parameters by
  tuning the following parameters: `learning_rate`, `l2_regularization`,
  `max_leaf_nodes`, and `min_samples_leaf`.

In [2]:
ordinal_encoding_columns = ['workclass', 'education', 'marital-status',
                            'occupation', 'relationship', 'race',
                            'native-country', 'sex']

categories = [data[column].unique()
              for column in data[ordinal_encoding_columns]]

preprocessor = ColumnTransformer(
    [('ordinal-encoder', OrdinalEncoder(categories=categories),
      ordinal_encoding_columns)],
    remainder='passthrough', sparse_threshold=0
)

model = Pipeline(
    [('preprocessor', preprocessor),
     ('gbrt', HistGradientBoostingClassifier(max_iter=50))]
)
param_distributions = {
    'gbrt__learning_rate': expon(loc=0.001, scale=0.5),
    'gbrt__l2_regularization': uniform(loc=0, scale=0.5),
    'gbrt__max_leaf_nodes': randint(5, 30),
    'gbrt__min_samples_leaf': randint(5, 30)
}
model_grid_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10, n_jobs=4
)
model_grid_search.fit(df_train, target_train)
print(
    f"The accuracy score using a {model_grid_search.__class__.__name__} is "
    f"{model_grid_search.score(df_test, target_test):.2f}"
)
print(f"The best set of parameters is: {model_grid_search.best_params_}")

The accuracy score using a RandomizedSearchCV is 0.88
The best set of parameters is: {'gbrt__l2_regularization': 0.4181258256605914, 'gbrt__learning_rate': 0.24206525417265057, 'gbrt__max_leaf_nodes': 22, 'gbrt__min_samples_leaf': 12}


In [3]:
df_results = pd.DataFrame(model_grid_search.cv_results_)
columns = (['mean_test_score', 'std_test_score'] +
           [col for col in df_results.columns if 'param_' in col])
df_results.sort_values(by='mean_test_score', ascending=False)[
    columns
]

,mean_test_score,std_test_score,param_gbrt__l2_regularization,param_gbrt__learning_rate,param_gbrt__max_leaf_nodes,param_gbrt__min_samples_leaf
9,0.869646,0.003524,0.418126,0.242065,22,12
8,0.869209,0.002566,0.46271,0.209745,18,28
0,0.868718,0.003084,0.122741,0.450102,8,24
1,0.868527,0.001893,0.482798,0.202828,17,19
7,0.865387,0.001892,0.366193,0.145731,10,18
3,0.863586,0.001984,0.101658,0.263843,5,10
5,0.855532,0.004291,0.041821,1.178118,28,10
2,0.846442,0.002653,0.381688,0.039852,5,23
6,0.845595,0.004577,0.389537,0.018332,22,12
4,0.817450,0.001878,0.053609,0.015692,13,7
